# LoRA from Scratch 

> see the original blog: https://lightning.ai/lightning-ai/studios/code-lora-from-scratch?section=featured&tab=overview 

## 1. LoRA From Scratch – Implement Low-Rank Adaptation for LLMs in PyTorch
LoRA, which stands for [Low-Rank Adaptation](https://arxiv.org/abs/2106.09685), is a popular technique to finetune LLMs more efficiently. Instead of adjusting all the parameters of a deep neural network, LoRA focuses on updating only a small set of low-rank matrices.

This Studio explains how LoRA works by coding it from scratch, which is an excellent exercise for looking under the hood of an algorithm.


## 2. Understanding LoRA
Pretrained LLMs are often dubbed foundation models because of their versatility across various tasks. However, it is often useful to adapt a pretrained LLM for a specific dataset or task, which we can accomplish via finetuning.

Finetuning allows the model to adapt to specific domains without costly pretraining, yet updating all layers can still be computationally expensive, especially for larger models.

LoRA offers a more efficient alternative to regular finetuning. As discussed in more detail in the paper [LoRA: Low-Rank Adaptation of Large Language Models](https://arxiv.org/abs/2106.09685), LoRA approximates a layer's weight changes during training, _**ΔW**,_ in a low-rank format.

For instance, whereas in regular finetuning, we compute the weight updates of a weight matrix _**W**_ as **_ΔW_**, in LoRA, we approximate **_ΔW_** through the matrix multiplication of two smaller matrices AB, as illustrated in the figure below. (If you are familiar with PCA or SVD, consider this as decomposing **_ΔW_** into **_A_** and **_B_**.)

For instance, whereas in regular finetuning, we compute the weight updates of a weight matrix _**W**_ as **_ΔW_**, in LoRA, we approximate **_ΔW_** through the matrix multiplication of two smaller matrices AB, as illustrated in the figure below. (If you are familiar with PCA or SVD, consider this as decomposing **_ΔW_** into **_A_** and **_B_**.)

<div style="text-align:center">
<img alt = "A comparison between the weight updates during the forward pass in regular finetuning (left) and LoRA (right)." src="../files/lora_01.png" width="900"/>
<em>A comparison between the weight updates during the forward pass in regular finetuning (left) and LoRA (right).</em></br></br>
</div>

Note that **_r_**, in the figure above, is a hyperparameter here that we can use to specify the rank of the low-rank matrices used for adaptation. A smaller **_r_** leads to a simpler low-rank matrix, which results in fewer parameters to learn during adaptation. This can lead to faster training and potentially reduced computational requirements. However, with a smaller **_r_**, the capacity of the low-rank matrix to capture task-specific information decreases.

To provide a concrete example, suppose the weight matrix W of a given layer has a size of 5,000x10,000 (50M parameters in total). If we choose a rank _**r=8**_, we initialize two matrices: the 5,000x8-dimensional matrix B and the 8x10,000-dimensional matrix A. Added together, A and B have only 80,000 + 40,000 = 120,000 parameters, which is 400 times smaller than the 50M parameters in regular finetuning via **_ΔW_**.

In practice, it’s important to experiment with different **_r_** values to find the right balance to achieve the desired performance on the new task.

## 3. Coding LoRA from Scratch

Since conceptual explanations can sometimes be abstract, let's now implement LoRA ourselves to get a better idea of how it works. In code, we can implement a LoRA layer as follows:
Here we implement LoRA as the following equation. A little abuse use of $A$ and $B$, opposite to the above figure $A$ and $B$.

$$ \begin{align*} Y &= X(W+\Delta W) = XW + XAB  \\
&= X_{b\times c_{\text{in}}} W_{c_{\text{in}} \times c_{\text{out}}} + X_{b\times c_{\text{in}}} A_{c_{\text{in}} \times r} B_{r \times c_{\text{out}}} 
\end{align*} \quad \text{with } r \ll min(c_{\text{in}}, c_{\text{out}})
$$

where, learnable parameters $A$ and $B$ are initialize as $A \sim \mathcal{N}(0,\, \sigma^{2})$ and $B=0$.

In [1]:
import torch

class LoRALayer(torch.nn.Module):
    def __init__(self, in_dim, out_dim, rank, alpha):
        super().__init__()
        std_dev = 1 / torch.sqrt(torch.tensor(rank).float())
        self.W_a = torch.nn.Parameter(torch.randn(in_dim, rank) * std_dev)
        self.W_b = torch.nn.Parameter(torch.zeros(rank, out_dim))
        self.alpha = alpha

    def forward(self, x):
        x = self.alpha * (x @ self.W_a @ self.W_b)
        return x

In [2]:
torch.manual_seed(123)

# a simple linear layer with 10 inputs and 1 output
# requires_grad=False makes it non-trainable
linear_layer = torch.nn.Linear(
    in_features = 10, out_features = 1, bias=True)

# a simple example input
x = torch.rand((1, 10))

linear_layer(x)

tensor([[-0.5745]], grad_fn=<AddmmBackward0>)

In the code above, `in_dim` is the input dimension of the layer we want to modify using LoRA, and `out_dim` is the respective output dimension of that layer.

We previously discussed that the rank of the matrices **_A_** and **_B_** (Note: here we use `self.W_a` and `self.W_b` for the matrices **_A_** and **_B_**, respectively) is a hyperparameter that controls the complexity and the number of additional parameters introduced by LoRA.

However, looking at the code above, we added another hyperparameter, the scaling factor `alpha`. This factor determines the magnitude of the changes introduced by the LoRA layer to the model's existing weights: `alpha * (x @ A @ B)`. A higher value of `alpha` means larger adjustments to the model's behavior, while a lower value results in more subtle changes.

Another thing to note is that we initialized `A` with small values from a random distribution. Here, the standard deviation of this distribution is determined by the square root of the rank (this choice ensures that the initial values in `A` are not too large.) However, we initialized `B` with zeros. The rationale here is that at the beginning of the training, before `A` and `B` are updated via backpropagation, the `LoRALayer` does not impact the original weights because _**AB=0**_ if **_B=0_**.

Note that LoRA is usually applied to a neural network's linear (feedforward) layers. For example, suppose we have a simple PyTorch model or module with two linear layers (e.g., this could be the feedforward module of a transformer block). And suppose this module's forward method looks like as follows:

In [3]:
class LinearWithLoRA(torch.nn.Module):
    def __init__(self, linear, rank, alpha):
        super().__init__()
        self.linear = linear
        self.lora = LoRALayer(
            linear.in_features, linear.out_features, rank, alpha
        )

    def forward(self, x):
        return self.linear(x) + self.lora(x)

Replace linear layer with LoRA layer:

In [4]:
lora_layer = LinearWithLoRA(linear=linear_layer, rank=8, alpha=1)
lora_layer(x)

tensor([[-0.5745]], grad_fn=<AddBackward0>)

Note that the LoRA layer will not change the linear layer output until it's trained because its `W_b` weight matrix is initialized to all zeros. For LoRA to take effect, we have to train the model. Below is a simple toy example.

Let's simulate a simple weight update:

In [5]:
lora_layer.lora.W_b = torch.nn.Parameter(lora_layer.lora.W_b + 0.01 * x[0])

We can now see that the output has changed:

In [6]:
lora_layer(x)

tensor([[-0.5863, -0.5758, -0.5779, -0.5800, -0.5814, -0.5766, -0.5887, -0.5811,
         -0.5859, -0.5892]], grad_fn=<AddBackward0>)

## 4. Finetuning with LoRA -- A Hands-On Example

### 1) Loading the dataset into DataFrames

In [7]:
import os
from datasets import load_dataset

import lightning as L
from lightning.pytorch.loggers import CSVLogger
from lightning.pytorch.callbacks import ModelCheckpoint

import pandas as pd
import torch

from local_dataset_utilities import download_dataset, load_dataset_into_to_dataframe, partition_dataset
from local_dataset_utilities import IMDBDataset

In [8]:
if not torch.cuda.is_available():
    print("Please switch to a GPU machine before running this notebook.")

In [9]:
files = ("test.csv", "train.csv", "val.csv")
download = True

for f in files:
    if not os.path.exists(os.path.join("data", f)):
        download = False

if download is False:
    download_dataset()
    df = load_dataset_into_to_dataframe()
    partition_dataset(df)

In [10]:
df_train = pd.read_csv(os.path.join("data", "train.csv"))
df_val = pd.read_csv(os.path.join("data", "val.csv"))
df_test = pd.read_csv(os.path.join("data", "test.csv"))

### 2) Tokenization and Numericalization

**Load the dataset via `load_dataset`**

In [11]:
imdb_dataset = load_dataset(
    "csv",
    data_files={
        "train": os.path.join("data", "train.csv"),
        "validation": os.path.join("data", "val.csv"),
        "test": os.path.join("data", "test.csv"),
    },
)

print(imdb_dataset)

DatasetDict({
    train: Dataset({
        features: ['index', 'text', 'label'],
        num_rows: 35000
    })
    validation: Dataset({
        features: ['index', 'text', 'label'],
        num_rows: 5000
    })
    test: Dataset({
        features: ['index', 'text', 'label'],
        num_rows: 10000
    })
})


**Tokenize the dataset**

In [12]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
print("Tokenizer input max length:", tokenizer.model_max_length)
print("Tokenizer vocabulary size:", tokenizer.vocab_size)

Tokenizer input max length: 512
Tokenizer vocabulary size: 30522


/home/changjiang/miniconda3/envs/py310_pt220/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [13]:
def tokenize_text(batch):
    return tokenizer(batch["text"], truncation=True, padding=True)

In [14]:
imdb_tokenized = imdb_dataset.map(tokenize_text, batched=True, batch_size=None)

Map:   0%|          | 0/35000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [15]:
del imdb_dataset

In [16]:
imdb_tokenized.set_format("torch", columns=["input_ids", "attention_mask", "label"])

In [17]:
os.environ["TOKENIZERS_PARALLELISM"] = "false"

### 3) Set Up DataLoaders

In [18]:
from torch.utils.data import DataLoader, Dataset


class IMDBDataset(Dataset):
    def __init__(self, dataset_dict, partition_key="train"):
        self.partition = dataset_dict[partition_key]

    def __getitem__(self, index):
        return self.partition[index]

    def __len__(self):
        return self.partition.num_rows

In [19]:
train_dataset = IMDBDataset(imdb_tokenized, partition_key="train")
val_dataset = IMDBDataset(imdb_tokenized, partition_key="validation")
test_dataset = IMDBDataset(imdb_tokenized, partition_key="test")

train_loader = DataLoader(
    dataset=train_dataset,
    batch_size=12,
    shuffle=True, 
    num_workers=4
)

val_loader = DataLoader(
    dataset=val_dataset,
    batch_size=12,
    num_workers=4
)

test_loader = DataLoader(
    dataset=test_dataset,
    batch_size=12,
    num_workers=4
)

### 4) Initializing DistilBERT

In [20]:
from transformers import AutoModelForSequenceClassification


model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=2)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


**Freeze all layers**

In [21]:
for param in model.parameters():
    param.requires_grad = False

**Add LoRA layers**

In [22]:
print ("Input model \n**************\n", model)

Input model 
**************
 DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dr

In [23]:
from functools import partial

lora_r = 8
lora_alpha = 16
lora_dropout = 0.05
lora_query = True
lora_key = False
lora_value = True
lora_projection = False
lora_mlp = False
lora_head = False

layers = []

assign_lora = partial(LinearWithLoRA, rank=lora_r, alpha=lora_alpha)

for layer in model.distilbert.transformer.layer:
    if lora_query:
        layer.attention.q_lin = assign_lora(layer.attention.q_lin)
    if lora_key:
        layer.attention.k_lin = assign_lora(layer.attention.k_lin)
    if lora_value:
        layer.attention.v_lin = assign_lora(layer.attention.v_lin)
    if lora_projection:
        layer.attention.out_lin = assign_lora(layer.attention.out_lin)
    if lora_mlp:
        layer.ffn.lin1 = assign_lora(layer.ffn.lin1)
        layer.ffn.lin2 = assign_lora(layer.ffn.lin2)
if lora_head:
    model.pre_classifier = assign_lora(model.pre_classifier)
    model.classifier = assign_lora(model.classifier)

In [24]:
print ("Now model + LoRA \n**************\n", model)

Now model + LoRA 
**************
 DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): LinearWithLoRA(
              (linear): Linear(in_features=768, out_features=768, bias=True)
              (lora): LoRALayer()
            )
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): LinearWithLoRA(
              (linear): Linear(in_features=768, out_features=768, bias=True)
              (lora): LoRALayer()
            )
            (out_lin): Linear(in_fe

**Let us check if linear layers are frozen**

In [25]:
# Check if linear layers are frozen
for name, param in model.named_parameters():
    print(f"{name}: {param.requires_grad}")

distilbert.embeddings.word_embeddings.weight: False
distilbert.embeddings.position_embeddings.weight: False
distilbert.embeddings.LayerNorm.weight: False
distilbert.embeddings.LayerNorm.bias: False
distilbert.transformer.layer.0.attention.q_lin.linear.weight: False
distilbert.transformer.layer.0.attention.q_lin.linear.bias: False
distilbert.transformer.layer.0.attention.q_lin.lora.W_a: True
distilbert.transformer.layer.0.attention.q_lin.lora.W_b: True
distilbert.transformer.layer.0.attention.k_lin.weight: False
distilbert.transformer.layer.0.attention.k_lin.bias: False
distilbert.transformer.layer.0.attention.v_lin.linear.weight: False
distilbert.transformer.layer.0.attention.v_lin.linear.bias: False
distilbert.transformer.layer.0.attention.v_lin.lora.W_a: True
distilbert.transformer.layer.0.attention.v_lin.lora.W_b: True
distilbert.transformer.layer.0.attention.out_lin.weight: False
distilbert.transformer.layer.0.attention.out_lin.bias: False
distilbert.transformer.layer.0.sa_layer_no

In [26]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


print("Total number of trainable parameters:", count_parameters(model))

Total number of trainable parameters: 147456


### 5) Finetuning

**Wrap in LightningModule for Training**

In [27]:
from local_model_utilities import CustomLightningModule

lightning_model = CustomLightningModule(model)

In [28]:
callbacks = [
    ModelCheckpoint(
        save_top_k=1, mode="max", monitor="val_acc"
    )  # save top 1 model
]
logger = CSVLogger(save_dir="logs/", name="my-model")

In [29]:
trainer = L.Trainer(
    max_epochs=3,
    callbacks=callbacks,
    accelerator="gpu",
    precision="16-mixed",
    devices=1,
    logger=logger,
    log_every_n_steps=10,
)

Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [30]:
import time
start = time.time()

trainer.fit(model=lightning_model,
            train_dataloaders=train_loader,
            val_dataloaders=val_loader)

end = time.time()
elapsed = end - start
print(f"Time elapsed {elapsed/60:.2f} min")

You are using a CUDA device ('NVIDIA GeForce RTX 3090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name     | Type                                | Params | Mode 
-------------------------------------------------------------------------
0 | model    | DistilBertForSequenceClassification | 67.1 M | eval 
1 | val_acc  | MulticlassAccuracy                  | 0      | train
2 | test_acc | MulticlassAccuracy                  | 0      | train
-------------------------------------------------------------------------
147 K     Trainable params
67.0 M    Non-trainable params
67.1 M    Total params
268.410   Total estimated model params size (MB)
26        Modules in train mode
96        M

Sanity Checking: |                                        | 0/? [00:00<?, ?it/s]

Training: |                                               | 0/? [00:00<?, ?it/s]

Validation: |                                             | 0/? [00:00<?, ?it/s]

Validation: |                                             | 0/? [00:00<?, ?it/s]

Validation: |                                             | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=3` reached.


Time elapsed 9.34 min


In [31]:
train_acc = trainer.test(lightning_model, dataloaders=train_loader, ckpt_path="best", verbose=False)
val_acc = trainer.test(lightning_model, dataloaders=val_loader, ckpt_path="best", verbose=False)
test_acc = trainer.test(lightning_model, dataloaders=test_loader, ckpt_path="best", verbose=False)

Restoring states from the checkpoint path at logs/my-model/version_0/checkpoints/epoch=1-step=5834.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loaded model weights from the checkpoint at logs/my-model/version_0/checkpoints/epoch=1-step=5834.ckpt
/home/changjiang/miniconda3/envs/py310_pt220/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:476: Your `test_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test dataloaders.


Testing: |                                                | 0/? [00:00<?, ?it/s]

Restoring states from the checkpoint path at logs/my-model/version_0/checkpoints/epoch=1-step=5834.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loaded model weights from the checkpoint at logs/my-model/version_0/checkpoints/epoch=1-step=5834.ckpt


Testing: |                                                | 0/? [00:00<?, ?it/s]

Restoring states from the checkpoint path at logs/my-model/version_0/checkpoints/epoch=1-step=5834.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loaded model weights from the checkpoint at logs/my-model/version_0/checkpoints/epoch=1-step=5834.ckpt


Testing: |                                                | 0/? [00:00<?, ?it/s]

In [34]:
print ("Finetuning with LoRA \n**************")
print(f"Train acc: {train_acc[0]['accuracy']*100:2.2f}%")
print(f"Val acc:   {val_acc[0]['accuracy']*100:2.2f}%")
print(f"Test acc:  {test_acc[0]['accuracy']*100:2.2f}%")
print ("********************************\n")

Finetuning with LoRA 
**************
Train acc: 93.58%
Val acc:   90.18%
Test acc:  89.63%
********************************



Cleanup checkpoint files as we don't need them later

In [35]:
import shutil

# Cleanup checkpoint files as we don't need them later
log_dir = f"logs/my-model"
if os.path.exists(log_dir):
    shutil.rmtree(log_dir)

## 5. Comparison to Traditional Finetuning

In the previous section, we obtained 89.44% test accuracy with LoRA default settings. How does this compare to traditional finetuning?

Let's start by training the DistilBERT model but only updating the last 2 layers during training. We can achieve this by first freezing all model weights and then unfreezing the two linear output layers:

In [36]:
del model
model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=2)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


**Freeze all layers**

In [37]:
print("Total number of trainable parameters:", count_parameters(model))

Total number of trainable parameters: 66955010


In [38]:
for param in model.parameters():
    param.requires_grad = False

**Unfreeze last layer**

In [39]:
print ("Input model \n**************\n", model)

Input model 
**************
 DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dr

In [40]:
for param in model.pre_classifier.parameters():
    param.requires_grad = True

for param in model.classifier.parameters():
    param.requires_grad = True

In [41]:
print("Total number of trainable parameters:", count_parameters(model))

Total number of trainable parameters: 592130


### Finetuning
- Wrap in LightningModule for Training, similar to the code shown above.

In [42]:
from local_model_utilities import CustomLightningModule

lightning_model = CustomLightningModule(model)

from lightning.pytorch.callbacks import ModelCheckpoint
from lightning.pytorch.loggers import CSVLogger


callbacks = [
    ModelCheckpoint(
        save_top_k=1, mode="max", monitor="val_acc"
    )  # save top 1 model
]
logger = CSVLogger(save_dir="logs/", name="my-model")

trainer = L.Trainer(
    max_epochs=3,
    callbacks=callbacks,
    accelerator="gpu",
    precision="16-mixed",
    devices=1,
    logger=logger,
    log_every_n_steps=10,
)

Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [43]:
import time
start = time.time()

trainer.fit(model=lightning_model,
            train_dataloaders=train_loader,
            val_dataloaders=val_loader)

end = time.time()
elapsed = end - start
print(f"Time elapsed {elapsed/60:.2f} min")

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name     | Type                                | Params | Mode 
-------------------------------------------------------------------------
0 | model    | DistilBertForSequenceClassification | 67.0 M | eval 
1 | val_acc  | MulticlassAccuracy                  | 0      | train
2 | test_acc | MulticlassAccuracy                  | 0      | train
-------------------------------------------------------------------------
592 K     Trainable params
66.4 M    Non-trainable params
67.0 M    Total params
267.820   Total estimated model params size (MB)
2         Modules in train mode
96        Modules in eval mode


Sanity Checking: |                                        | 0/? [00:00<?, ?it/s]

Training: |                                               | 0/? [00:00<?, ?it/s]

Validation: |                                             | 0/? [00:00<?, ?it/s]

Validation: |                                             | 0/? [00:00<?, ?it/s]

Validation: |                                             | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=3` reached.


Time elapsed 4.29 min


In [44]:
test_acc = trainer.test(lightning_model, dataloaders=test_loader, ckpt_path="best", verbose=False)

Restoring states from the checkpoint path at logs/my-model/version_0/checkpoints/epoch=2-step=8751.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loaded model weights from the checkpoint at logs/my-model/version_0/checkpoints/epoch=2-step=8751.ckpt


Testing: |                                                | 0/? [00:00<?, ?it/s]

In [45]:
print ("finetuning with last 2 layers\n****************")
print(f"Train acc: {train_acc[0]['accuracy']*100:2.2f}%")
print(f"Val acc:   {val_acc[0]['accuracy']*100:2.2f}%")
print(f"Test acc:  {test_acc[0]['accuracy']*100:2.2f}%")
print ("****************\n")

finetuning with last 2 layers
****************
Train acc: 93.58%
Val acc:   90.18%
Test acc:  86.45%
****************



In [46]:
import shutil

# Cleanup checkpoint files as we don't need them later
log_dir = f"logs/my-model"
if os.path.exists(log_dir):
    shutil.rmtree(log_dir)

So, as we can see, finetuning all layers, which involves training 66,955,010 parameters, performs better than both finetuning only the last two layers (592,130 parameters) and the LoRA defaults (147,456 parameters).

As a takeaway message, so far, LoRA performs better than the conventional finetuning of the last two layers even though it uses 4x fewer parameters. Finetuning all layers requires updating 450x more parameters than in the LoRA setting but also results in 2% higher test accuracy. 

However, one aspect to consider is that we only used LoRA default settings so far. Perhaps we can bridge the gap between the full finetuning and LoRA finetuning with a different LoRA hyperparameter configuration? We will answer this question in the upcoming section.

## 6. Optimizing the LoRA Configuration

The previous LoRA finetuning results were based on the following hyperparameter choices:

```python
lora_r = 8
lora_alpha = 16
lora_dropout = 0.05
lora_query = True
lora_key = False
lora_value = True
lora_projection = False
lora_mlp = False
lora_head = False
```

Note that this only involves applying LoRA to an attention layer's query and value weight matrices. Optionally, we can enable LoRA for other layers as well. Furthermore, we can control the number of trainable parameters in each LoRA layer by modifying the rank (`lora_r`). 

To try different hyperparameter configurations, you can use our compact `finetune-lora-script.py` script, which accepts the hyperparameter choices as command line arguments:

```bash
python finetune-lora-script.py --lora_alpha 32 --lora_r 16
```

In addition, you can also toggle other hyperparameter settings, for example:

```bash
python 03_finetune-lora.py \
--lora_alpha 32 \
--lora_r 16 \
--lora_query True \
--lora_key True \
--lora_value True \
--lora_projection True \
--lora_mlp True \
--lora_head True
```

One way to improve the LoRA performance is to manually fiddle with these hyperparameter choices. However, to make the hyperparameter tuning more convenient, you can also use the `gridsearch.py` script, which runs the following hyperparameter grid on all available GPUs:

```python
alpha_values = [1, 4, 8, 16, 32, 64]
rank_values = [1, 2, 4, 8, 16, 32]
devices = range(torch.cuda.device_count())
lora_query = ["True"]
lora_key = ["False", "True"]
lora_value = ["True"]
lora_projection = ["False", "True"]
lora_mlp = ["False", "True"]
lora_head = ["False", "True"]
```

The way the `finetune-lora-script.py` script is set up, the results are saved to a `results.txt file`. Inspecting the `results.txt` file after the run is completed, the best hyperparameter configurations seem to be the following:

```plain
lora_r: 8
lora_alpha: 1
lora_query: True
lora_key: False
lora_value: True
lora_projection: False
lora_mlp: True
lora_head: False
```

Resulting in:
-   Val acc: 92.96%
-   Test acc: 92.39%

Let us run the grid search code. It will actually call the python scirpt `finetune-lora-script.py` as shown in the function `run_script()` defined in the `gridsearch.py`.

```python
def run_script(alpha, rank, device, query, key, value, projection, mlp, head):
    global device_usage
    # it will call the python srcipt `finetune-lora-script.py`;
    command = [
        'python3', 'finetune-lora-script.py',
        '--lora_alpha', str(alpha),
        '--lora_r', str(rank),
        '--device', str(device),
        '--lora_query', query,
        '--lora_key', key,
        '--lora_value', value,
        '--lora_projection', projection,
        '--lora_mlp', mlp,
        '--lora_head', head,
        '--verbose', "False"
    ]

    print(f"Starting run with alpha = {alpha}, rank = {rank}, lora_query = {query}, lora_key = {key}, lora_value = {value}, lora_projection = {projection}, lora_mlp = {mlp}, lora_head = {head} on device {device}")
    subprocess.run(command)
    print(f"Completed run with alpha = {alpha}, rank = {rank}, lora_query = {query}, lora_key = {key}, lora_value = {value}, lora_projection = {projection}, lora_mlp = {mlp}, lora_head = {head} on device {device}")

    # Mark the device as no longer in use
    device_usage[device] = False
```